In [1]:
# Exercises:
# E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?
# E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?
# E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?
# E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?
# E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?
# E06: meta-exercise! Think of a fun/interesting exercise and complete it.

## E01

In [1]:
# loading the data
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn.functional as F


words = open("names.txt", 'r').read().splitlines()

# words des
print("length : ",len(words))
print("max word length : ",max(len(w) for w in words))
print("min word length : ",min(len(w) for w in words))
print(words[:5])

length :  32033
max word length :  15
min word length :  2
['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [2]:
# preprocessing
N = torch.zeros((27,729), dtype=torch.int32)

a = '.abcdefghijklmnopqrstuvwxyz'
d_stoi = {}
count = 0
for i in a:
    for j in a:
        d_stoi[(i+j)] = count
        count += 1

s_stoi = {}
count = 0
for i in a:
    s_stoi[i] = count
    count += 1
s_stoi

# reverse stoi
d_itos = {i:s for s,i in d_stoi.items()}
s_itos = {i:s for s,i in s_stoi.items()}



for w in words:
    w = '.' + w + '.'
    length = len(w)
    if length > 1:
        for i in range(length - 1): 
            ix1 = d_stoi[(w[i:i+2])]
            try: 
                ix2 = s_stoi[(w[i+2])]
            except:
                ix2 = s_stoi['.']
            N[ix2, ix1] += 1

In [3]:
out = []
P = (N+1).float()
row_sum = P.sum(1, keepdim=True) 

P /= row_sum
P[0].sum()

tensor(1.)

In [4]:
# loss
log_likelihood = 0.0
counter = 0

for w in words:
    w = '.' + w + '.'
    length = len(w)
    if length > 1:
        for i in range(length - 1): 
            ix1 = d_stoi[(w[i:i+2])]
            try: 
                ix2 = s_stoi[(w[i+2])]
            except:
                ix2 = s_stoi['.']
            N[ix2, ix1] += 1
            prob = P[ix2, ix1]
            logprob = torch.log(prob)
            log_likelihood += logprob
            counter +=1



negative_log_likelihood = -log_likelihood
avg_negative_log_likelihood = negative_log_likelihood / counter

print(f'{log_likelihood=}')
print(f'{negative_log_likelihood=}')
print(f'{avg_negative_log_likelihood=}')

log_likelihood=tensor(-1041502.6875)
negative_log_likelihood=tensor(1041502.6875)
avg_negative_log_likelihood=tensor(4.5651)


In [25]:
d_stoi

{'..': 0,
 '.a': 1,
 '.b': 2,
 '.c': 3,
 '.d': 4,
 '.e': 5,
 '.f': 6,
 '.g': 7,
 '.h': 8,
 '.i': 9,
 '.j': 10,
 '.k': 11,
 '.l': 12,
 '.m': 13,
 '.n': 14,
 '.o': 15,
 '.p': 16,
 '.q': 17,
 '.r': 18,
 '.s': 19,
 '.t': 20,
 '.u': 21,
 '.v': 22,
 '.w': 23,
 '.x': 24,
 '.y': 25,
 '.z': 26,
 'a.': 27,
 'aa': 28,
 'ab': 29,
 'ac': 30,
 'ad': 31,
 'ae': 32,
 'af': 33,
 'ag': 34,
 'ah': 35,
 'ai': 36,
 'aj': 37,
 'ak': 38,
 'al': 39,
 'am': 40,
 'an': 41,
 'ao': 42,
 'ap': 43,
 'aq': 44,
 'ar': 45,
 'as': 46,
 'at': 47,
 'au': 48,
 'av': 49,
 'aw': 50,
 'ax': 51,
 'ay': 52,
 'az': 53,
 'b.': 54,
 'ba': 55,
 'bb': 56,
 'bc': 57,
 'bd': 58,
 'be': 59,
 'bf': 60,
 'bg': 61,
 'bh': 62,
 'bi': 63,
 'bj': 64,
 'bk': 65,
 'bl': 66,
 'bm': 67,
 'bn': 68,
 'bo': 69,
 'bp': 70,
 'bq': 71,
 'br': 72,
 'bs': 73,
 'bt': 74,
 'bu': 75,
 'bv': 76,
 'bw': 77,
 'bx': 78,
 'by': 79,
 'bz': 80,
 'c.': 81,
 'ca': 82,
 'cb': 83,
 'cc': 84,
 'cd': 85,
 'ce': 86,
 'cf': 87,
 'cg': 88,
 'ch': 89,
 'ci': 90,
 'cj': 91

In [34]:
P[0][1:27]

tensor([1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05,
        1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05,
        1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05,
        1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05, 1.5433e-05,
        1.5433e-05, 1.5433e-05])

In [13]:
torch.multinomial(P[0], num_samples=1, replacement=True).item()


329

In [20]:
P[0].shape

torch.Size([729])

In [39]:
torch.multinomial(P[:,329], num_samples=1, replacement=True).item()

20

In [35]:
# sampling
ix = 0
for i in range(10):
    name =''
    while True:
        ix = torch.multinomial(P[ix], num_samples=1, replacement=True).item()
        name +=d_itos[ix]
        ix = torch.multinomial(P[:,ix], num_samples=1, replacement=True).item()
        name += s_itos[ix]
        if ix == 0:  #if its find '.'
            break

    out.append(name)
print(out)

['i..', 'h..', 'd.cmav.amziclexnesetptoroukscftaxeumvcqwilauoavsdirlfwluclulheoddpardandhavcstniliya.koasvricnvbvqkicenyskankasespqdjuntkim.ajygbsanmayddwindowyrlo.luhopblqwyjmlqrxqguzazzous.isjamkipzetrihaksruwdejeno.luvapmau.yuhanaiqyhqnso.ju.haavqqaxfzj.faezxeztfaqvdtatysha.nykamsulnysaiddalcobyfzas.', 'lvebrfmrjlinorfhirmbyalabrjrejijjugeinajamladlyniamyaniahispsapayfritrebempcou.ryksvdcwbxxere.roiyaendsiyrlybriknjm.wfuqrux.odi.gromiahmak.nokraurieliloseuksaveuqpjcjacinnwih.ko.ma.motrvaragildawnpzjob.adarspafncwromlajszjehkuydfkqsumlatmikmyr.abhyasrxciu.lotsqssimeecha.ju.juehjzuljuwdhrdar.arkynrisricelsari.xzwppher.ch.dqruvalu.ni.cocloricnkpqhfwebnajspu.sknijqmq.dwntzcwcaldinanna.yoaviasftejha.', 'a..', 'cky.whetz.im.evdzqellmikirk.sh.su.amlenampjtzhvqdiaama.jukayembsabeclzrkladjacintandinerye.brhab.anawqwgqequriauwqv..', 'ahxtcu.qwyhuloucur.brmeehwzaylck.', 'a..', 'e..', 'ie.']
